In [53]:
import os
import re
import pickle
from tqdm import tqdm

import torch

from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer, util

In [2]:
# gpu setting
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"  # Arrange GPU devices starting from 0
os.environ["CUDA_VISIBLE_DEVICES"] = "1"  # Set the GPU 0 to use

In [3]:
base_path = os.getcwd()
data_path = os.path.join(base_path, 'data')
data_path

'/home/heerak/workspace/universal-triggers/data'

In [4]:
positive_word = []
negative_word = []
with open(os.path.join(data_path, 'SentiWord_Dict.txt'), 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines, start=1):
        if i <= 43:  # 이모티콘 제거(상담 데이터일 것이므로 저런 이모티콘은 생성되지 않을 듯)
            continue
        split_line = line.split()
        word = ' '.join(split_line[:-1])
        if re.match(r'[ㅏ-ㅣ]+.*[ㅏ-ㅣ]*', word):
            continue
        score = int(split_line[-1])
        if score > 0:
            positive_word.append(word)
        elif score < 0:
            negative_word.append(word)

with open(os.path.join(data_path, 'negative-words.txt'), 'r', encoding="ISO-8859-1") as f:
    lines = f.readlines()
    for i, line in enumerate(lines, start=1):
        if i <= 30:
            continue
        word = line.strip()
        negative_word.append(word)

with open(os.path.join(data_path, 'positive-words.txt'), 'r') as f:
    lines = f.readlines()
    for i, line in enumerate(lines, start=1):
        if i <= 30:
            continue
        word = line.strip()
        positive_word.append(word)

In [5]:
positive_word[:5], positive_word[-5:], len(positive_word)

(['가격이 싸다', '가까이 사귀어', '가까이하다', '가능성이 늘어나다', '가능성이 있다고'],
 ['youthful', 'zeal', 'zenith', 'zest', 'zippy'],
 6853)

In [6]:
negative_word[:5], negative_word[-5:], len(negative_word)

(['가꾸러뜨리다', '가꾸러트리다', '가난', '가난뱅이', '가난살이'],
 ['zaps', 'zealot', 'zealous', 'zealously', 'zombie'],
 14588)

In [7]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-base')

In [86]:
vocab = tokenizer.vocab
word2idx = {k: v for k, v in sorted(vocab.items(), key=lambda x: x[1])}
idx2word = {v: k for k, v in word2idx.items()}
vocab_list = [k for k, v in word2idx.items()]
# # 또는 [ 이 들어있는 토큰 index list
remove_vocab_set = set([word2idx[vocab] for vocab in vocab_list if re.match(r'[#]+', vocab) or '[' in vocab])
vocab_list[:5], len(vocab_list), len(remove_vocab_set)

(['[CLS]', '[PAD]', '[SEP]', '[UNK]', '[MASK]'], 32000, 5318)

In [71]:
preprocessed_vocab_list = [f'내 기분은 {vocab} 상태야' for vocab in vocab_list]

In [72]:
st_model = SentenceTransformer('Huffon/sentence-klue-roberta-base')

In [73]:
if not os.path.exists(os.path.join(base_path, 'embeddings')):
    os.makedirs(os.path.join(base_path, 'embeddings'), exist_ok=True)
    
if not os.path.exists(os.path.join(base_path, 'embeddings', 'klue-roberta-base-vocab-embeddings.pkl')):
    embeddings = st_model.encode(preprocessed_vocab_list, convert_to_tensor=True).cpu()
    with open(os.path.join(base_path, 'embeddings', 'klue-roberta-base-vocab-embeddings.pkl'), 'wb') as f:
        pickle.dump(embeddings, f)
else:
    with open(os.path.join(base_path, 'embeddings', 'klue-roberta-base-vocab-embeddings.pkl'), 'rb') as f:
        embeddings = pickle.load(f)

In [74]:
embeddings, len(embeddings), len(embeddings[0])

(tensor([[-0.5526, -0.3601,  0.4703,  ...,  0.3581, -0.1404,  0.2446],
         [-0.4787, -0.1982,  0.5745,  ...,  0.2828,  0.2304,  0.3660],
         [-0.5139, -0.3661,  0.7755,  ...,  0.4761, -0.0692,  0.3479],
         ...,
         [ 0.1333,  0.6229,  0.2930,  ...,  0.3645,  0.0465,  0.3907],
         [-0.1236,  0.5605,  0.2219,  ...,  0.4005, -0.1580,  0.3047],
         [-0.0424,  0.5613,  0.2530,  ...,  0.3898, -0.1984,  0.2144]]),
 32000,
 768)

In [97]:
positive_token_index = set()
for pos_word in tqdm(positive_word):
    query = f'내 기분은 {pos_word} 상태야'
    # query = f'내 기분은 어이없는 상태야'
    
    query_embedding = st_model.encode(query)
    cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]

    threshold = 0.7
    while torch.where(cosine_scores > threshold)[0].size(0) >= 100:
        threshold += 0.05
    
    num = torch.where(cosine_scores > threshold)[0].size(0)
    for vocab_idx in cosine_scores.argsort(descending=True)[:num]:
        if cosine_scores[vocab_idx] <= threshold:
            break
        if int(vocab_idx) not in remove_vocab_set and int(vocab_idx) not in positive_token_index:
            print(f'index: {vocab_idx:>5}\t{query} <> {preprocessed_vocab_list[vocab_idx]}\tScore: {cosine_scores[vocab_idx]:.4f}')
            positive_token_index.add(int(vocab_idx))

  0%|          | 2/6853 [00:00<08:42, 13.12it/s]

index:  1338	내 기분은 가격이 싸다 상태야 <> 내 기분은 싼 상태야	Score: 0.8151
index:  3852	내 기분은 가격이 싸다 상태야 <> 내 기분은 가격 상태야	Score: 0.7536
index: 14453	내 기분은 가격이 싸다 상태야 <> 내 기분은 반값 상태야	Score: 0.7179
index:  7498	내 기분은 가격이 싸다 상태야 <> 내 기분은 원가 상태야	Score: 0.7125
index: 28336	내 기분은 가까이 사귀어 상태야 <> 내 기분은 사귄 상태야	Score: 0.8730
index: 11679	내 기분은 가까이 사귀어 상태야 <> 내 기분은 사귀 상태야	Score: 0.8449
index: 14379	내 기분은 가까이 사귀어 상태야 <> 내 기분은 친해 상태야	Score: 0.8207
index: 15371	내 기분은 가까이 사귀어 상태야 <> 내 기분은 교제 상태야	Score: 0.8095
index: 13809	내 기분은 가까이 사귀어 상태야 <> 내 기분은 친밀 상태야	Score: 0.7686
index:  7784	내 기분은 가까이 사귀어 상태야 <> 내 기분은 연인 상태야	Score: 0.7401
index: 25703	내 기분은 가까이 사귀어 상태야 <> 내 기분은 친밀감 상태야	Score: 0.7388
index: 13268	내 기분은 가까이 사귀어 상태야 <> 내 기분은 친한 상태야	Score: 0.7386
index:  7738	내 기분은 가까이 사귀어 상태야 <> 내 기분은 연애 상태야	Score: 0.7333
index:  9433	내 기분은 가까이 사귀어 상태야 <> 내 기분은 밀접 상태야	Score: 0.7200
index:  6291	내 기분은 가까이 사귀어 상태야 <> 내 기분은 만남 상태야	Score: 0.7176
index:  3949	내 기분은 가까이 사귀어 상태야 <> 내 기분은 친구 상태야	Score: 0.7033
index: 28548	내 기분은 가까이 사귀어 상

  0%|          | 6/6853 [00:00<08:26, 13.51it/s]

index:  8518	내 기분은 가능성이 늘어나다 상태야 <> 내 기분은 늘어날 상태야	Score: 0.7866
index: 23460	내 기분은 가능성이 늘어나다 상태야 <> 내 기분은 무궁무진 상태야	Score: 0.7202
index: 10227	내 기분은 가능성이 늘어나다 상태야 <> 내 기분은 열릴 상태야	Score: 0.7167
index: 11695	내 기분은 가능성이 늘어나다 상태야 <> 내 기분은 증가세 상태야	Score: 0.7118
index:  3964	내 기분은 가능성이 늘어나다 상태야 <> 내 기분은 증가 상태야	Score: 0.7118
index:  3662	내 기분은 가능성이 있다고 상태야 <> 내 기분은 가능 상태야	Score: 0.8392
index: 27908	내 기분은 가능성이 있다고 상태야 <> 내 기분은 가망 상태야	Score: 0.7741
index:   861	내 기분은 가능하다 상태야 <> 내 기분은 될 상태야	Score: 0.8718
index:  4894	내 기분은 가능하다 상태야 <> 내 기분은 된다는 상태야	Score: 0.8687
index: 12151	내 기분은 가능하다 상태야 <> 내 기분은 이루어진다 상태야	Score: 0.8592
index: 13718	내 기분은 가능하다 상태야 <> 내 기분은 해낼 상태야	Score: 0.8530
index:  9733	내 기분은 가능하다 상태야 <> 내 기분은 이뤄질 상태야	Score: 0.8526
index: 16560	내 기분은 가능하다 상태야 <> 내 기분은 원한다면 상태야	Score: 0.8511


  0%|          | 8/6853 [00:00<08:22, 13.62it/s]

...

index:  4148	내 기분은 works 상태야 <> 내 기분은 업무 상태야	Score: 0.8004
index:  7299	내 기분은 world-famous 상태야 <> 내 기분은 전당 상태야	Score: 0.7059
index: 21343	내 기분은 worth 상태야 <> 내 기분은 워드 상태야	Score: 0.7119
index:     8	내 기분은 worth 상태야 <> 내 기분은 $ 상태야	Score: 0.7094
index: 23439	내 기분은 worth 상태야 <> 내 기분은 너비 상태야	Score: 0.7058
index: 20517	내 기분은 worth 상태야 <> 내 기분은 양구 상태야	Score: 0.7032


100%|█████████▉| 6849/6853 [08:17<00:00, 14.05it/s]

index: 29736	내 기분은 wows 상태야 <> 내 기분은 위스 상태야	Score: 0.8161
index:    61	내 기분은 yay 상태야 <> 내 기분은 Y 상태야	Score: 0.8695


100%|██████████| 6853/6853 [08:17<00:00, 13.77it/s]

index:  1547	내 기분은 zenith 상태야 <> 내 기분은 젠 상태야	Score: 0.7568
index:  1440	내 기분은 zenith 상태야 <> 내 기분은 옌 상태야	Score: 0.7558
index: 17839	내 기분은 zenith 상태야 <> 내 기분은 아지 상태야	Score: 0.7543
index:    62	내 기분은 zest 상태야 <> 내 기분은 Z 상태야	Score: 0.8595
index: 15117	내 기분은 zippy 상태야 <> 내 기분은 지기 상태야	Score: 0.7617
index:  5407	내 기분은 zippy 상태야 <> 내 기분은 지자 상태야	Score: 0.7522


In [100]:
negative_token_index = set()
for neg_word in tqdm(negative_word):
    query = f'내 기분은 {neg_word} 상태야'
    # query = f'내 기분은 어이없는 상태야'
    
    query_embedding = st_model.encode(query)
    cosine_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]

    threshold = 0.7
    while torch.where(cosine_scores > threshold)[0].size(0) >= 100:
        threshold += 0.05
    
    num = torch.where(cosine_scores > threshold)[0].size(0)
    for vocab_idx in cosine_scores.argsort(descending=True)[:num]:
        if cosine_scores[vocab_idx] <= threshold:
            break
        if int(vocab_idx) not in remove_vocab_set and int(vocab_idx) not in negative_token_index:
            print(f'index: {vocab_idx:>5}\t{query} <> {preprocessed_vocab_list[vocab_idx]}\tScore: {cosine_scores[vocab_idx]:.4f}')
            negative_token_index.add(int(vocab_idx))

  0%|          | 2/14588 [00:00<18:01, 13.49it/s]

index: 29965	내 기분은 가꾸러뜨리다 상태야 <> 내 기분은 꾸밀 상태야	Score: 0.7764
index: 12890	내 기분은 가꾸러뜨리다 상태야 <> 내 기분은 손질 상태야	Score: 0.7349
index: 26767	내 기분은 가꾸러뜨리다 상태야 <> 내 기분은 기른 상태야	Score: 0.7149
index: 28693	내 기분은 가꾸러뜨리다 상태야 <> 내 기분은 꾸며진 상태야	Score: 0.7126
index: 13653	내 기분은 가꾸러뜨리다 상태야 <> 내 기분은 키운 상태야	Score: 0.7011
index: 11412	내 기분은 가꾸러뜨리다 상태야 <> 내 기분은 다듬 상태야	Score: 0.7002
index: 22833	내 기분은 가꾸러트리다 상태야 <> 내 기분은 원예 상태야	Score: 0.7045
index:  6192	내 기분은 가난 상태야 <> 내 기분은 가난 상태야	Score: 1.0000
index:  8986	내 기분은 가난 상태야 <> 내 기분은 빈곤 상태야	Score: 0.8392
index: 23194	내 기분은 가난 상태야 <> 내 기분은 궁핍 상태야	Score: 0.8300
index: 21053	내 기분은 가난 상태야 <> 내 기분은 빈곤층 상태야	Score: 0.7846
index: 16942	내 기분은 가난 상태야 <> 내 기분은 빈민 상태야	Score: 0.7566
index:  9494	내 기분은 가난 상태야 <> 내 기분은 저소득층 상태야	Score: 0.7542
index: 27863	내 기분은 가난 상태야 <> 내 기분은 생활고 상태야	Score: 0.7194
index:  9644	내 기분은 가난 상태야 <> 내 기분은 열악 상태야	Score: 0.7116
index:  8150	내 기분은 가난 상태야 <> 내 기분은 저소득 상태야	Score: 0.7069


  0%|          | 10/14588 [00:00<17:14, 14.10it/s]

index: 27402	내 기분은 가난에 상태야 <> 내 기분은 재정난 상태야	Score: 0.7001
index: 18649	내 기분은 가난하게 상태야 <> 내 기분은 불우 상태야	Score: 0.7311
index: 20605	내 기분은 가난하게 상태야 <> 내 기분은 박한 상태야	Score: 0.7027
index: 13857	내 기분은 가난하게 상태야 <> 내 기분은 초라 상태야	Score: 0.7010


  0%|          | 12/14588 [00:00<17:25, 13.94it/s]

...

100%|█████████▉| 14579/14588 [17:35<00:00, 15.33it/s]

index: 12504	내 기분은 writhe 상태야 <> 내 기분은 저술 상태야	Score: 0.8131
index: 10243	내 기분은 writhe 상태야 <> 내 기분은 글쓰기 상태야	Score: 0.8110
index:  4197	내 기분은 writhe 상태야 <> 내 기분은 작가 상태야	Score: 0.7899
index: 21435	내 기분은 writhe 상태야 <> 내 기분은 회고록 상태야	Score: 0.7802
index:  7577	내 기분은 writhe 상태야 <> 내 기분은 원고 상태야	Score: 0.7772
index: 24092	내 기분은 writhe 상태야 <> 내 기분은 극작 상태야	Score: 0.7656
index: 11145	내 기분은 writhe 상태야 <> 내 기분은 집필 상태야	Score: 0.7578
index: 19110	내 기분은 writhe 상태야 <> 내 기분은 글귀 상태야	Score: 0.7554
index: 21343	내 기분은 writhe 상태야 <> 내 기분은 워드 상태야	Score: 0.7538
index: 14269	내 기분은 writhe 상태야 <> 내 기분은 열람 상태야	Score: 0.7507


100%|█████████▉| 14583/14588 [17:35<00:00, 14.23it/s]

index:    61	내 기분은 yawn 상태야 <> 내 기분은 Y 상태야	Score: 0.8312
index:    62	내 기분은 zap 상태야 <> 내 기분은 Z 상태야	Score: 0.7862
index:  1576	내 기분은 zap 상태야 <> 내 기분은 즈 상태야	Score: 0.7505


100%|██████████| 14588/14588 [17:36<00:00, 13.81it/s]


In [101]:
len(vocab_list), len(positive_token_index), len(negative_token_index)

(32000, 12266, 15441)

In [102]:
with open(os.path.join(data_path, 'sentiment_token_index', 'remove_token.pkl'), 'wb') as f:
    pickle.dump(remove_vocab_set, f)
with open(os.path.join(data_path, 'sentiment_token_index', 'positive.pkl'), 'wb') as f:
    pickle.dump(positive_token_index, f)
with open(os.path.join(data_path, 'sentiment_token_index', 'negative.pkl'), 'wb') as f:
    pickle.dump(negative_token_index, f)